In [19]:
# 경로를 지정해주자.
import os 

path = "c:\\x-ray"
annot_path = os.path.join(path,"annotations")
img_path = os.path.join(path,"images")
label_path = os.path.join(path,"labels")

import xml.etree.ElementTree as ET
import glob
import os
import json

# xml bbox 형식을 yolo bbox 형태로 변환하는 함수
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]
    
classes = []

from tqdm import tqdm

files = glob.glob(os.path.join(annot_path, '*.xml'))
for fil in tqdm(files):
    
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    
    result = []
    
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
    if result:
        with open(os.path.join(label_path, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

  0%|          | 0/2125 [00:00<?, ?it/s]

100%|██████████| 2125/2125 [00:13<00:00, 155.87it/s]


In [20]:
# 데이터세트를 train/valid로 분리하고 각각의 txt 만들기
import os 
import glob

path = "c:\\x-ray"
annot_path = os.path.join(path,"annotations")
img_path = os.path.join(path,"images")
label_path = os.path.join(path,"labels")

data = glob.glob(os.path.join(img_path,"*.png"))
train = data[:1700] # 2125 * 0.8 
valid = data[1701:]

print(len(train), len(valid))

# train.txt
with open(os.path.join(path, "train.txt"), 'w') as f:
	f.write('\n'.join(train) + '\n')

# valid.txt
with open(os.path.join(path, "valid.txt"), 'w') as f:
	f.write('\n'.join(valid) + '\n')

# -------- yaml에 경로 지정하기 

import yaml

yaml_data = {"names":['LapTop', 'TabletPC', 'SmartPhone', 'USB', 'SSD', 'HDD', 'HDD_External'], # 클래스 이름
             "nc":7, # 클래스 수
             "path":path, # root 경로
             "train":os.path.join(path, "train.txt"), # train.txt 경로
             "val":os.path.join(path, "valid.txt"), # valid.txt 경로
             }

with open(os.path.join(path, "data.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

1700 424


In [21]:
import os
path = 'c:\\x-ray\\labels'  # path of labels
labels = os.listdir(path)
for x in labels:
    with open(os.path.join(path, x)) as f:
        lines = f.read().splitlines()
        for y in lines:
          try:
            if int(y[:2])>6: #if your last label is 2 digit ie nc=7 [0-6] label  or change slice value 
              print(x)      # it will print .txt file name
          except:
            pass

In [22]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [23]:
%time
%cd c:\\yolov5

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

!python train.py --img 640 --batch 16 --epochs 10 --data C:/x-ray/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name xray_results --device 0 --cache

CPU times: total: 0 ns
Wall time: 0 ns
c:\yolov5


train: weights='', cfg=./models/custom_yolov5s.yaml, data=C:/x-ray/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=xray_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 10 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-294-gdb125a20 Python-3.8.19 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentu

In [24]:
%time
%cd c:\\yolov5

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

!python train.py --img 640 --batch 16 --epochs 30 --data C:/x-ray/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name xray_results --device 0 --cache

CPU times: total: 0 ns
Wall time: 1e+03 µs
c:\yolov5


train: weights='', cfg=./models/custom_yolov5s.yaml, data=C:/x-ray/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=xray_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 10 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-294-gdb125a20 Python-3.8.19 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentu

In [4]:
%time
%cd c:\\yolov5

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

!python train.py --img 640 --batch 16 --epochs 50 --data C:/x-ray/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name xray_results --device 0 --cache

CPU times: total: 0 ns
Wall time: 0 ns
c:\yolov5


train: weights='', cfg=./models/custom_yolov5s.yaml, data=C:/x-ray/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=xray_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-294-gdb125a20 Python-3.8.19 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_moment

In [3]:
%time
%cd c:\\yolov5

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

!python val.py --weights runs/train/xray_results_single_other_epoch10/weights/best.pt --data C:/x-ray/data.yaml --img 640 --iou 0.65 --half --name xray_results --device 0

CPU times: total: 0 ns
Wall time: 0 ns
c:\yolov5


c:\Users\user\anaconda3\envs\yolov5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
val: data=C:/x-ray/data.yaml, weights=['runs/train/xray_results_single_other_epoch50/weights/best.pt'], batch_size=32, imgsz=10, conf_thres=0.001, iou_thres=0.5, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=xray_results, exist_ok=False, half=True, dnn=False
YOLOv5  v7.0-294-gdb125a20 Python-3.8.19 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
WARNING  --img-size 10 must be multiple of max stride 32, updating to 32

val: Scanning C:\x-ray\valid.cache... 424 images, 0 b

In [14]:
!python detect.py --weights runs/train/xray_results_multiple_other_epoch30/weights/best.pt --img 640 --conf 0.4 --source C:\x-ray\images

detect: weights=['runs/train/xray_results_multiple_other_epoch30/weights/best.pt'], source=C:\x-ray\images, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-294-gdb125a20 Python-3.8.19 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/541 C:\x-ray\images\S_8471.30-0000_01_271.png: 384x640 1 USB, 2 SmartPhones, 10.9ms
image 2/541 C:\x-ray\images\S_8471.30-0000_01_272.png: 384x640 1 USB, 1 SmartPhone, 16.8ms
image 3/541 C:\x-ray\images\S_8471.30-0000_01_273.png: 384x640 1 USB, 1 SmartPhone